In [1]:
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import random
import pandas as pd

In [2]:
movie_titles = []
genre = []
certifications = []
director = []
stars = []
year = []
runtime = []
votes = []
metascore = []
imdb_rating = []
gross = []

In [3]:
pages = np.arange(1,3001,50)

In [4]:
for page in pages:
    page = requests.get('https://www.imdb.com/search/title/?title_type=feature,tv_movie,documentary,short&colors=color&sort=num_votes,desc&start='+str(page)+'&ref_=adv_nxt')
    soup = BeautifulSoup(page.text, 'html.parser')
   
    movie_data = soup.find_all('div',{'class':'lister-item mode-advanced'})
    
    for movie in movie_data:
        
        #title of movie
        title = movie.h3.a.text
        movie_titles.append(title)
        
        #year of movie release and removing parentheses
        year_release = movie.h3.find('span', {'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','').replace('V','').replace('I','').replace('X','').strip()
        year.append(year_release)
        
        #how long each movie was and removing 'min' and white spaces
        duration = movie.p.find('span',{'class':'runtime'}).text.replace('min','').strip()
        runtime.append(duration)

        #votes/gross'in millions'
        values = movie.find_all('span',{'name':'nv'})
        voting = values[0].text.replace(',','')
        votes.append(voting)

        if len(values) > 1:
            profit = values[1].text.replace('M','').replace('$','')
            gross.append(profit)
        else:
            null = None
            gross.append(null)

        #metascore
        if movie.find('div',{'class':'inline-block ratings-metascore'}):
            score = movie.find('div',{'class':'inline-block ratings-metascore'}).text.replace('\n','').replace('Metascore','').strip()
            metascore.append(score)
        else:
            null = None
            metascore.append(null)    

        #rating
        rating = movie.find('div',{'class':'inline-block ratings-imdb-rating'}).text.replace('\n','')
        imdb_rating.append(float(rating))

        #type of movie 
        movie_type = movie.p.find('span',{'class':'genre'}).text.replace('\n','').strip()
        genre.append(movie_type)


        #content rating 
        if movie.p.find('span',{'class':'certificate'}):
            content = movie.p.find('span',{'class':'certificate'}).text
            if content == 'Not Rated':
                content = None

            certifications.append(content) 
        else:
            null = None
            certifications.append(null)

        #director/stars of movie 
        cast = movie.find('p',class_='').text.replace('\n','').replace('Director:','').replace('Directors:','').replace('Stars:','').split('|')
        cast[0],cast[1] = [person.strip() for person in cast]
        director.append(cast[0])
        stars.append(cast[1])

In [5]:
features = [movie_titles,imdb_rating, genre, certifications, director, stars, year, gross, runtime, votes, metascore]
headers = ['Title','IMDB Rating','Genre','Content Rating','Director','Stars', 'Release Year','Domestic Gross Earnings','Runtime','Votes','Metascore']


In [6]:
imdb_dict = dict(zip(headers,features))

In [7]:
imdb = pd.DataFrame.from_dict(imdb_dict, orient='columns')

In [8]:
imdb.to_csv('imdb_movies.csv',header=True,index=False)